In [ ]:
# rag(retrieval augmented generation): 검색 증강 기법

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader, UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI(temperature=0.1)

cache_dir = LocalFileStore("../.cache/")


# # chunk_size는 토큰의 길이를 제한 / chunk_overlap은 앞 부분 일부분을 겹치게 만들게 합니다.
# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=100,
#     chunk_overlap=10
# )

# 아래는 separator가 있습니다. 
# RecursiveCharacterTextSplitter의 기능을 모두 갖고 있으면서 seperator가 있기 때문에 해당 방법을 추천합니다. 
# 토큰화 방법에 대한 직관적인 방법을 보고 싶으면 해당 경로를 참고하세요
# https://platform.openai.com/tokenizer

splitter = CharacterTextSplitter(
    separator = '\n',
    chunk_size = 600,
    chunk_overlap= 50,
)

# 해당 loader는 pdf, txt, docx와 모두 호환됩니다.
loader = UnstructuredFileLoader("../files/_241117_chapter_one.txt")
docs = loader.load_and_split(text_splitter=splitter)

# embedding에 대한 직관적인 이해가 필요하다면 다음 링크를 참고한다.
# https://turbomaze.github.io/word2vecjson/
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)
# FAISS와 CHROMA를 변경할 수 있다ㅣ
# vectorstore = Chroma.from_documents(docs[:10], cached_embeddings)
vectorstore = FAISS.from_documents(docs[:10], cached_embeddings)

# chain type: stuff, refine, 
# UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
# map_reduce와 map_rerank는 depricated되어서 오류를 내보냅니다. 
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='refine',
    retriever=vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions")

Created a chunk of size 963, which is longer than the specified 600
Created a chunk of size 774, which is longer than the specified 600
Created a chunk of size 954, which is longer than the specified 600
Created a chunk of size 922, which is longer than the specified 600
Created a chunk of size 1168, which is longer than the specified 600
Created a chunk of size 821, which is longer than the specified 600
Created a chunk of size 700, which is longer than the specified 600
Created a chunk of size 745, which is longer than the specified 600
Created a chunk of size 735, which is longer than the specified 600
Created a chunk of size 1110, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 990, which is longer than the specified 600
Created a chunk of size 1182, which is longer than the specified 600
Created a chunk of size 1491, which is longer than the specified 600
Created a chunk of size 1401, which is longe

Created a chunk of size 930, which is longer than the specified 600
Created a chunk of size 1022, which is longer than the specified 600
Created a chunk of size 1260, which is longer than the specified 600
Created a chunk of size 795, which is longer than the specified 600
Created a chunk of size 1293, which is longer than the specified 600
Created a chunk of size 649, which is longer than the specified 600


"Victory Mansions is a building where Winston Smith resides, located in a run-down and grim area of London. The living conditions in Victory Mansions are poor, with the hallway smelling of boiled cabbage and old rag mats. The building is adorned with a large poster depicting the face of Big Brother, a figurehead of the oppressive government in George Orwell's dystopian world. The atmosphere of Victory Mansions is further emphasized by the presence of the nearby Ministry of Truth, one of the four massive government buildings that dominate the skyline. The Ministry of Truth, along with the Ministries of Peace, Love, and Plenty, collectively govern all aspects of society in this totalitarian regime."